In [1]:
import os
import sys
# import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sb
from scipy import stats
# from keras.wrappers.scikit_learn import KerasClassifier
from IPython.display import display, HTML

HACKATHON_DATA_PATH = "C:\Hackathon_Data"

In [2]:
complaints_df = pd.read_csv('C:\Hackathon_Data\OFT Data files redacted\OFT Data files redacted\Fair Trading builder complaints data.csv', encoding='ISO-8859-1', error_bad_lines=False)
builder_df = pd.read_csv('C:\Hackathon_Data\PII_Redacted_Extract_Files_20181203\R_BUILDER_EXPORT.txt', encoding='ISO-8859-1', error_bad_lines=False)
policy_df = pd.read_csv('C:\Hackathon_Data\PII_Redacted_Extract_Files_20181203\R_POLICY_EXPORT.txt', encoding='ISO-8859-1', error_bad_lines=False)
elig_df = pd.read_csv('C:\Hackathon_Data\PII_Redacted_Extract_Files_20181203\R_ELIGIBILITY_EXPORT.txt', encoding='ISO-8859-1', error_bad_lines=False)

complaints_df['Licence'] = complaints_df['Licence'].str.lstrip()
complaints_df = complaints_df[complaints_df['Licence'].notna()]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15,54,57,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 197673: expected 109 fields, saw 112\n'
b'Skipping line 365536: expected 109 fields, saw 110\n'
b'Skipping line 414650: expected 109 fields, saw 111\n'
b'Skipping line 424887: expected 109 fields, saw 111\nSkipping line 424888: expected 109 fields, saw 111\n'
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,2,3,4,5,6,9,10,11,13,17,39,45,48,49,67,68,73,74,76,77,82,86,87,89,90,91,92,93,94,95,96,98,100,101,102,103,104,106,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
pd.set_option('display.max_columns', None)

elig_df = elig_df[elig_df['current_elig'].notna()]
elig_df = elig_df[elig_df['current_elig'].str.contains('Y')]

print('Merging tables...')
complaints_builder_df = complaints_df.merge(builder_df, left_on='Licence', right_on='builder_bk', how='left')
policy_builder_df = policy_df.merge(builder_df, left_on='builder_bk', right_on='builder_bk', how='left')
builder_df = elig_df.merge(builder_df, left_on='builder_licence_number', right_on='builder_bk', how='left')

claims_df = pd.read_csv('C:\Hackathon_Data\PII_Redacted_Extract_Files_20181203\R_CLAIM_EXPORT.txt', encoding = "ISO-8859-1", error_bad_lines=False )
policy_with_claim = list(claims_df['policy_bk'].values)
policy_builder_df['Has_Claim'] = policy_builder_df['policy_bk'].isin(policy_with_claim).astype(int)

print(policy_builder_df.shape)

Merging tables...


In [ ]:
print('Converting Date Formats')
from pprint import pprint

def parse_policy_dates(series):
    series = series.astype(str).replace('\.0', '', regex=True)
    return pd.to_datetime(series, format='%Y%m%d', errors='coerce')

def get_numeric_stats(series):
    mean = np.mean(series[series.notna()])
    std = np.std(series[series.notna()])
    skew = stats.skew(series[series.notna()])
    kurtosis = stats.kurtosis(series[series.notna()])
#     print('{}: mean:{:.2f} std:{:.2f}'.format(series.name, mean, std))
    return mean, std, skew, kurtosis

def unpack_value_counts(series, prefix=''):
    counts = series.value_counts(normalize=True, sort=True, ascending=False, bins=None, dropna=True)
    
    out_dict = {}
    for k, v in counts.to_dict().items():
        new_key = '{}{}_{}_proportion'.format(prefix, series.name, k).lower().replace(' ', '_').replace('\\', '_')
        out_dict[new_key] = v
    return out_dict

def normalize(df, columns_to_normalize):
    result = df.copy()
    for feature_name in columns_to_normalize:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
    
policy_builder_df['issue_date_parsed'] = parse_policy_dates(policy_builder_df['issue_date'])
policy_builder_df['act_completion_date_parsed'] = parse_policy_dates(policy_builder_df['act_completion_date'])
policy_builder_df['est_completion_date_parsed'] = parse_policy_dates(policy_builder_df['est_completion_date'])

policy_builder_df['est_project_duration'] = (policy_builder_df['est_completion_date_parsed'] - policy_builder_df['issue_date_parsed']).dt.days
policy_builder_df['act_project_duration'] = (policy_builder_df['act_completion_date_parsed'] - policy_builder_df['issue_date_parsed']).dt.days

# List of dictionaries
builders_data = []
builders_complaints_temporal = []

NUM_BUILDERS_TO_PROCESS = 500
# NUM_BUILDERS_TO_PROCESS = -1

num_total = len(policy_builder_df['builder_bk'][policy_builder_df['builder_bk'].notna()].unique())

for count, unique_id in enumerate(policy_builder_df['builder_bk'][policy_builder_df['builder_bk'].notna()].unique()[:NUM_BUILDERS_TO_PROCESS]):
    print('Aggregating complaints and policy data for builder {}/{}'.format(count, num_total))
    builder_data = {'builder_bk': unique_id}
        
    # Get info for this builder
    masked_builder_df = policy_builder_df[policy_builder_df['builder_bk'] == unique_id]
    
    builder_categorical_cols = ['builder_entity_type',
                                'builder_review_size',
                                'builder_review_type',
                                'primary_builder_segment_code',
                                'secondary_builder_segment_code']
    
#     display(masked_builder_df)
    for colname in builder_categorical_cols:
        builder_data[colname] = masked_builder_df[colname].iloc[0]
    
    # Aggregate the policy data for this builder
    masked_policy_builder_df = policy_builder_df[policy_builder_df['builder_bk'] == unique_id]
    builder_data['Has_Claim'] = masked_policy_builder_df['Has_Claim'].any()
    
    # Numeric policy Data
    numeric_cols = ['est_project_duration',
                    'act_project_duration',
                    'contract_amount',
                    'approvedjobnumberlimit',
                    'approvedjobnumbervalue']
   
    for colname in numeric_cols:
        mean, std, skew, kurtosis = get_numeric_stats(masked_policy_builder_df[colname])
        builder_data['mean_{}'.format(colname)] = mean
        builder_data['std_{}'.format(colname)] = std
        builder_data['skew_{}'.format(colname)] = skew
        builder_data['kurtosis_{}'.format(colname)] = kurtosis
    
    # Categorical policy Data
    categorical_cols = ['cover_type',
#         'security_type',
        'relationship_to_builder',
        'speculative_project',
        'architectordesigner_tendered',
        'builder_licence_status_code']
    
    for colname in categorical_cols:
        builder_data.update(unpack_value_counts(masked_policy_builder_df[colname]))
            

    # Aggregate the complaint data for this builder
    masked_complaint_builder_df = complaints_builder_df[complaints_builder_df['builder_bk'] == unique_id]
    masked_complaint_builder_df['Date'] = pd.to_datetime(masked_complaint_builder_df['Registered Date'], format='%d/%m/%y %H:%M')
    masked_complaint_builder_df = masked_complaint_builder_df[['Goods', 'Product', 'Practice', 'Date', 'Customer PC']].sort_values(by='Date')
    
    complaint_counts = normalize(masked_complaint_builder_df[['Date', 'Practice']].groupby(pd.Grouper(key='Date', freq='M')).count(), ['Practice'])
    builder_complaints_temporal = complaint_counts.transpose()
#     print(unpack_value_counts(complaint_counts))
#     try:
#         ax = complaint_counts.plot(legend=False)
#         ax.set_ylabel('Normalised number of complaints')
#         ax.set_title('Temporal histogram of complaints for {}'.format(unique_id))
#     except TypeError:
#         print('No complaints for this builder')

    # Categorical complaint data
    builder_data.update(unpack_value_counts(masked_complaint_builder_df['Practice'], prefix='complaint_'))
    builder_data.update(unpack_value_counts(masked_complaint_builder_df['Product'], prefix='complaint_'))
            
    builders_data.append(pd.DataFrame([builder_data], columns=builder_data.keys()))
    builders_complaints_temporal.append(builder_complaints_temporal)
    
builder_features_df = pd.concat(builders_data, axis=0).reset_index()
builder_temporal_features_df = pd.concat(builders_complaints_temporal, axis=0).reset_index().fillna(0.0)
builder_temporal_features_df = builder_temporal_features_df.add_prefix('complaints_')

print(builder_temporal_features_df)
                                                      
# Fill NaN proportions with zero!
cols_to_fill = [col for col in builder_features_df if col.endswith('_proportion')]
builder_features_df[cols_to_fill] = builder_features_df[cols_to_fill].fillna(0.0)

# display(builder_features_df)
print('Saving PKL')
builder_features_df.to_pickle('builder_features_df_test.pkl')

In [ ]:
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

builder_categorical_cols = ['builder_entity_type',
                            'builder_review_size',
                            'builder_review_type',
                            'primary_builder_segment_code',
                            'secondary_builder_segment_code']

builder_features_encoded_df = builder_features_df

for col in builder_categorical_cols:
    le = preprocessing.LabelEncoder()
    builder_features_encoded_df[col] = le.fit_transform(builder_features_df[col].fillna(''))

y = builder_features_encoded_df['Has_Claim']
X = builder_features_encoded_df.drop(['builder_bk', 'Has_Claim'], axis=1)

X = X.apply(lambda x: x.fillna(x.mean())) 

display(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['rbf']}

print(X.shape, y.shape)
# grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2, n_jobs=-1, scoring='f1')
# # estimator = RandomForestClassifier()
# # selector = RFE(estimator, 25, step=1)
# print('Fitting!')
# # selector = selector.fit(X, y)
# grid.fit(X, y)
# pprint(grid.grid_scores_)
# print('Done SVC!')

# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 4)]
# # Number of features to consider at every split
# max_features = ['sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 3)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf}

# grid = GridSearchCV(RandomForestClassifier(), random_grid, refit = True, verbose=2, n_jobs=-1, scoring='f1')
# estimator = RandomForestClassifier(max_depth=60, max_features='sqrt', min_samples_leaf=1, min_samples_split=5, n_estimators=2000)
estimator = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
# estimator = KNeighborsClassifier(n_neighbors=5, algorithm='auto')
# estimator = QuadraticDiscriminantAnalysis()
# estimator = RFE(estimator, 25, step=1)
print('Fitting!')
# selector = selector.fit(X, y)
estimator.fit(X_train, y_train)
y_pred = estimator.predict(X_test)
print(classification_report(y_test, y_pred))
print(f1_score(y_test, y_pred, average='binary'))
print('micro', f1_score(y_test, y_pred, average='micro'))
print('macro', f1_score(y_test, y_pred, average='macro'))
print('weighted', f1_score(y_test, y_pred, average='weighted'))
print('Done!')

In [ ]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(8,4))
# ax1 = sb.countplot(x='builder_entity_type', data=builder_df)
# plt.figure(figsize=(20,4))
# ax2 = sb.countplot(x='builder_review_size', data=builder_df)
# plt.figure(figsize=(8,4))
# ax3 = sb.countplot(x='builder_review_type', data=builder_df)
# plt.figure(figsize=(20,4))
# ax4 = sb.countplot(x='primary_builder_segment_code', data=builder_df)
# plt.figure(figsize=(20,4))
# ax5 = sb.countplot(x='secondary_builder_segment_code', data=builder_df)
# plt.figure(figsize=(8,4))
# ax = sb.countplot(x='cover_type', data=policy_builder_df)
# plt.figure(figsize=(8,4))
# ax = sb.countplot(x='security_type', data=policy_builder_df)
# plt.figure(figsize=(8,4))
# ax = sb.countplot(x='relationship_to_builder', data=policy_builder_df)
# plt.figure(figsize=(8,4))
# ax = sb.countplot(x='speculative_project', data=policy_builder_df)
# plt.figure(figsize=(8,4))
# ax = sb.countplot(x='architectordesigner_tendered', data=policy_builder_df)
# plt.figure(figsize=(8,4))
# ax = sb.countplot(x='builder_licence_status_code', data=policy_builder_df)

# for i in range(1,9):
#     plt.figure(figsize=(8,4))
#     ax = sb.countplot(x='c0{}_cover_yn'.format(i), data=builder_df)